## Cleaning up Atlas data
**Function**      : Preprocess netCDF files and restructure the dataset<br>
**Author          : Team BETA**<br>
**First Built**   : 2021.08.11<br>
**Last Update     : 2021.08.11**<br>
**Library**       : os, numpy, netcdf4, xarray<br>
**Description**   : In this notebook serves to clean up Atlas data which is given in netcdf format and aggregate the data into a single file.<br>
**Return Values   : .nc files**<br>
**Note**          : All the data is saved to netCDF4 format. Note that data from different models may vary concerning the resolution and coordinates.<br>

In [2]:
import os
import xarray as xr

### Path
Specify the path to the dataset and the place to save the outputs. <br>

In [3]:
# please specify data path
datapath = '/mnt/d/NLeSC/BETA/EUCP/Atlas'
# please specify output path
output_path = '../assets/sample_figures/test'
os.makedirs(output_path, exist_ok = True)

### Extract data
Extract weather/climate data from given netCDF files.

In [5]:
# CNRS
# model unknown
dataset = xr.open_dataset(os.path.join(datapath,'Rita_Lukas_Atlas',
                          'eur_CMIP5_pr_2041-2060_vs_1995-2014_10_JJA.nc'))
dataset

<xarray.Dataset>
Dimensions:        (lon: 20, lat: 17)
Coordinates:
  * lon            (lon) float64 -8.75 -6.25 -3.75 -1.25 ... 33.75 36.25 38.75
  * lat            (lat) float64 31.25 33.75 36.25 38.75 ... 66.25 68.75 71.25
Data variables:
    pr_weighted    (lat, lon) float64 ...
    pr_unweighted  (lat, lon) float64 ...
Attributes:
    CDI:          Climate Data Interface version 1.9.7.1 (http://mpimet.mpg.d...
    Conventions:  CF-1.6
    history:      Sat Jun 19 10:31:00 2021: cdo merge pr_CMIP5_p10_w_JJA.nc p...
    NCO:          netCDF Operators version 4.7.4 (http://nco.sf.net)
    CDO:          Climate Data Operators version 1.9.7.1 (http://mpimet.mpg.d...

# Peter's mess

In [1]:
import xarray as xr

In [20]:
# combining multiple variables into one dataset
tas = xr.open_dataset('/home/peter/eucp-project/atlas_data/REA_Ben/eur_CMIP5_tas_2041-2060_vs_1995-2014_10_DJF.nc')
pr = xr.open_dataset('/home/peter/eucp-project/atlas_data/REA_Ben/eur_CMIP5_pr_2041-2060_vs_1995-2014_10_DJF.nc')
ds = xr.merge([tas, pr])
ds

<xarray.Dataset>
Dimensions:         (lon: 20, lat: 17)
Coordinates:
  * lon             (lon) float64 -8.75 -6.25 -3.75 -1.25 ... 33.75 36.25 38.75
  * lat             (lat) float64 31.25 33.75 36.25 38.75 ... 66.25 68.75 71.25
    height          float64 ...
Data variables:
    tas_weighted    (lat, lon) float64 ...
    tas_unweighted  (lat, lon) float64 ...
    pr_weighted     (lat, lon) float64 ...
    pr_unweighted   (lat, lon) float64 ...
Attributes:
    CDI:          Climate Data Interface version 1.9.7.1 (http://mpimet.mpg.d...
    Conventions:  CF-1.6
    history:      Sat Jun 19 10:31:08 2021: cdo merge tas_CMIP5_p10_w_DJF.nc ...
    NCO:          netCDF Operators version 4.7.4 (http://nco.sf.net)
    CDO:          Climate Data Operators version 1.9.7.1 (http://mpimet.mpg.d...

In [32]:
# Combining a dataset along multiple dimensions
tas_10p = xr.open_dataset('/home/peter/eucp-project/atlas_data/REA_Ben/eur_CMIP5_tas_2041-2060_vs_1995-2014_10_DJF.nc').drop('height').assign_coords(percentile=10)
tas_90p = xr.open_dataset('/home/peter/eucp-project/atlas_data/REA_Ben/eur_CMIP5_tas_2041-2060_vs_1995-2014_90_DJF.nc').drop('height').assign_coords(percentile=90)
ds = xr.concat([tas_10p, tas_90p], dim='percentile')
ds

<xarray.Dataset>
Dimensions:         (lon: 20, lat: 17, percentile: 2)
Coordinates:
  * lon             (lon) float64 -8.75 -6.25 -3.75 -1.25 ... 33.75 36.25 38.75
  * lat             (lat) float64 31.25 33.75 36.25 38.75 ... 66.25 68.75 71.25
  * percentile      (percentile) int64 10 90
Data variables:
    tas_weighted    (percentile, lat, lon) float64 1.188 1.535 1.404 ... nan nan
    tas_unweighted  (percentile, lat, lon) float64 1.188 1.511 1.396 ... nan nan
Attributes:
    CDI:          Climate Data Interface version 1.9.7.1 (http://mpimet.mpg.d...
    Conventions:  CF-1.6
    history:      Sat Jun 19 10:31:08 2021: cdo merge tas_CMIP5_p10_w_DJF.nc ...
    NCO:          netCDF Operators version 4.7.4 (http://nco.sf.net)
    CDO:          Climate Data Operators version 1.9.7.1 (http://mpimet.mpg.d...

In [35]:
# Combining many percentiles in a loop
datasets = []
for perc in [10, 25, 50, 75, 90]:
    ds = xr.open_dataset(f'/home/peter/eucp-project/atlas_data/REA_Ben/eur_CMIP5_tas_2041-2060_vs_1995-2014_{perc}_DJF.nc').drop('height').assign_coords(percentile=perc)
    datasets.append(ds)
    
ds = xr.concat(datasets, dim='percentile')
ds

<xarray.Dataset>
Dimensions:         (lon: 20, lat: 17, percentile: 50)
Coordinates:
  * lon             (lon) float64 -8.75 -6.25 -3.75 -1.25 ... 33.75 36.25 38.75
  * lat             (lat) float64 31.25 33.75 36.25 38.75 ... 66.25 68.75 71.25
Dimensions without coordinates: percentile
Data variables:
    tas_weighted    (percentile, lat, lon) float64 1.188 1.535 1.404 ... nan nan
    tas_unweighted  (percentile, lat, lon) float64 1.188 1.511 1.396 ... nan nan
Attributes:
    CDI:          Climate Data Interface version 1.9.7.1 (http://mpimet.mpg.d...
    Conventions:  CF-1.6
    history:      Sat Jun 19 10:31:08 2021: cdo merge tas_CMIP5_p10_w_DJF.nc ...
    NCO:          netCDF Operators version 4.7.4 (http://nco.sf.net)
    CDO:          Climate Data Operators version 1.9.7.1 (http://mpimet.mpg.d...

In [136]:
# Combining multiple dimensions with a preprocessor
def add_percentile(ds):
    filename = ds.encoding["source"]
    percentile = int(filename.split('_')[-2])
    dataset, variable, future, _, reference, percentile, season = filename.split('_')[-7:]
    return(ds
           .drop_vars('height', errors='ignore')
           .assign_coords(percentile=int(percentile)).expand_dims('percentile')
          )

# ds = xr.open_mfdataset('/home/peter/eucp-project/atlas_data/REA_Ben/eur_CMIP5_tas_2041-2060_vs_1995-2014_*_DJF.nc', preprocess=add_coords)
# ds

def load_data(project, season, variable):
    ds = xr.open_mfdataset(f'/home/peter/eucp-project/atlas_data/REA_Ben/eur_{project}_{variable}_2041-2060_vs_1995-2014_*_{season}.nc', preprocess=add_percentile)
    weighted = ds[f'{variable}_weighted'].rename(variable).assign_coords(constrained=1).expand_dims('constrained')
    unweighted = ds[f'{variable}_unweighted'].rename(variable).assign_coords(constrained=0).expand_dims('constrained')    
    return xr.concat([weighted, unweighted], dim='constrained')
    

projects = {}
for project in ['CMIP5', 'CMIP6', 'CORDEX']:
    seasons = []
    for season in ['DJF', 'JJA']:
        tas = load_data(project, season, 'tas')
        pr = load_data(project, season, 'pr')
        ds = xr.merge([tas, pr]).assign_coords(season=season)
        seasons.append(ds)
    ds = xr.concat(seasons, dim='season')
    ds.to_netcdf(f'cleaned_REA_{project}')

In [142]:
ds = xr.open_dataset('cleaned_REA_CMIP5')
import hvplot.xarray
app = ds.hvplot.quadmesh(cmap='coolwarm', coastline=True)
app

:DynamicMap   [season,constrained,percentile]
   :Overlay
      .QuadMesh.I  :QuadMesh   [lon,lat]   (tas)
      .Coastline.I :Feature   [Longitude,Latitude]

In [144]:
import panel as pn
pn.Row(app).save('atlas.html', embed=True)